In [19]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [22]:
df = pd.read_csv('./mhealth_raw_data.csv')   # https://www.kaggle.com/datasets/gaurav2022/mobile-health/data
min_size = int((df['Activity'].value_counts().min()))   # balanhandle imbalanced data labels
stratified_sample = df.groupby('Activity').apply(lambda x: x.sample(n=min_size, random_state=42))
stratified_sample.drop('subject', axis=1, inplace=True)
stratified_sample

alx      aly      alz      glx      gly       glz  \
Activity                                                                  
0        1212350  1.33590  -9.7394 -1.66110  0.58813 -0.58724 -0.662080   
         134482   2.52730 -12.7320 -3.88220  0.51763 -0.58537 -0.758350   
         820343   0.55205  -9.7466  1.58070  0.55288 -0.76173 -0.176820   
         718210   2.49960  -9.3680  1.81700  0.73655 -0.55159  0.068762   
         118798  -3.48370  -9.2520  0.54423 -0.37662 -0.61163  0.609040   
...                   ...      ...      ...      ...      ...       ...   
12       717006   4.21450  -3.1419 -9.24000  0.39147 -0.89493  0.013752   
         716463   5.36410  -6.1145 -8.48130  0.53432 -0.81051 -0.007858   
         716662   0.70801 -19.3040 -4.62480  0.48237 -0.82739  0.080550   
         114268   0.20464  -2.3600 -3.32430  0.72913 -0.56098 -0.392930   
         977665  -2.79510 -19.0980 -8.81620 -0.45455 -0.48968 -0.913560   

                      arx       ary      arz       grx       gry      grz  \
Activity                                                                    
0        1212350  -3.2703  -9.05310 -0.30682  0.401960 -1.061600  0.14655   
         134482   -4.3463 -10.51300  1.71390 -0.272550 -0.765910  0.85129   
         820343   -5.8155  -8.16430 -2.42900  0.137250 -0.997950  0.52155   
         718210   -3.0504  -5.91350  4.65220  0.084314 -0.710470  0.90086   
         118798   -6.5952  -0.79894  7.10590 -0.900000  1.145800  0.81897   
...                   ...       ...      ...       ...       ...      ...   
12       717006  -20.8850 -18.81200 -8.24270 -0.856860 -0.375770  0.56897   
         716463  -21.1400 -18.69800 -0.77864 -0.694120 -0.710470  0.47629   
         716662   -2.4445  -1.29750 -1.75240 -0.956860 -0.207390  0.48491   
         114268    3.7371   0.66210 -2.78980 -0.547060  0.289530  0.93966   
         977665  -21.2660  -3.68250 -0.24662 -0.666670  0.016427 -0.78879   

                  Activity  
Activity                    
0        1212350         0  
         134482          0  
         820343          0  
         718210          0  
         118798          0  
...                    ...  
12       717006         12  
         716463         12  
         716662         12  
         114268         12  
         977665         12  

[134446 rows x 13 columns]

In [23]:
X = stratified_sample.drop('Activity', axis=1)
y = stratified_sample['Activity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((107556, 12), (26890, 12), (107556,), (26890,))

In [24]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

scaler.mean_, scaler.scale_

(array([ 1.70971205, -9.09947274, -0.84459523,  0.08074532, -0.56302103,
        -0.1370391 , -3.53950139, -5.66709452,  2.31681035, -0.23059068,
        -0.41651389,  0.3579665 ]),
 array([4.43386527, 5.29852172, 6.57840717, 0.46723938, 0.41742006,
        0.55514724, 6.06001735, 6.60135923, 4.2558025 , 0.54293752,
        0.54414737, 0.52820823]))

## Model

In [6]:
nn = Sequential([
    Dense(128, activation='relu', input_shape=(12,)),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(48, activation='relu'),
    Dense(24, activation='relu'),
    Dense(13, activation='softmax')
])

nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1664      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 48)                3120      
                                                                 
 dense_4 (Dense)             (None, 24)                1176      
                                                                 
 dense_5 (Dense)             (None, 13)                325       
                                                                 
Total params: 18,701
Trainable params: 18,701
Non-traina

2024-07-06 20:39:12.641906: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:
nn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn.fit(X_train_scale, y_train, epochs=15, batch_size=32)

Epoch 1/15
3362/3362 [==============================] - 13s 4ms/step - loss: 0.5857 - accuracy: 0.8065
Epoch 2/15
3362/3362 [==============================] - 12s 4ms/step - loss: 0.3291 - accuracy: 0.8940
Epoch 3/15
3362/3362 [==============================] - 12s 4ms/step - loss: 0.2771 - accuracy: 0.9119
Epoch 4/15
3362/3362 [==============================] - 14s 4ms/step - loss: 0.2503 - accuracy: 0.9213
Epoch 5/15
3362/3362 [==============================] - 16s 5ms/step - loss: 0.2308 - accuracy: 0.9277
Epoch 6/15
3362/3362 [==============================] - 13s 4ms/step - loss: 0.2174 - accuracy: 0.9317
Epoch 7/15
3362/3362 [==============================] - 13s 4ms/step - loss: 0.2052 - accuracy: 0.9358
Epoch 8/15
3362/3362 [==============================] - 13s 4ms/step - loss: 0.1960 - accuracy: 0.9392
Epoch 9/15
3362/3362 [==============================] - 12s 4ms/step - loss: 0.1873 - accuracy: 0.9431
Epoch 10/15
3362/3362 [==============================] - 13s 4ms/step - l

In [13]:
loss_nn, acc_nn = nn.evaluate(X_test_scale, y_test)
predictions_nn = nn.predict(X_test_scale)
predictions_classes = np.argmax(predictions_nn, axis=1)
prec_nn = precision_score(y_test, predictions_classes, average='macro')
recall_nn = recall_score(y_test, predictions_classes, average='macro')
f1_nn = f1_score(y_test, predictions_classes, average='macro')

print('Neural Network Accuracy is: ', acc_nn)
print('Neural Network Precision is ', prec_nn)
print('Neural Network Recall is ', recall_nn)
print('Neural Network F1 Score is ', f1_nn)

841/841 [==============================] - 2s 2ms/step
Neural Network Accuracy is:  0.9414280652999878
Neural Network Precision is  0.9389324215585337
Neural Network Recall is  0.9413783304414992
Neural Network F1 Score is  0.9390572882457857


In [14]:
confusion_matrix(y_test, predictions_classes)

array([[1298,   66,   41,   34,   80,  100,   64,   90,  138,   62,   25,
          24,   56],
       [   0, 2035,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   0,    0, 2090,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [   0,    0,    0, 2105,    0,    0,    0,    0,    0,    0,    0,
           0,    0],
       [  68,    7,    0,    0, 2000,    4,    3,    2,    2,    0,    0,
           0,    2],
       [  79,    0,    0,    0,   24, 1874,    9,    1,   38,    0,    0,
           1,    2],
       [   7,    2,    0,    0,    0,    0, 2063,   16,    9,    0,    0,
           0,    0],
       [   4,    1,    0,    0,    0,    0,   16, 2084,   15,    0,    0,
           0,    0],
       [   6,    0,    0,    0,    2,    1,    4,    7, 2027,    1,    0,
           0,    0],
       [  35,    0,    0,    0,    0,    0,    0,    3,    0, 2034,    0,
           0,    0],
       [  30,    0,    0,    0,    0,    1,    0, 

## Export our model for tflite
We need to convert our model into a tflite model

In [15]:
converter = tf.lite.TFLiteConverter.from_keras_model(nn)
tflite_quant_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmp5gg1xj4o/assets


INFO:tensorflow:Assets written to: /tmp/tmp5gg1xj4o/assets
2024-07-06 20:44:25.884665: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-07-06 20:44:25.884702: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-07-06 20:44:25.885064: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp5gg1xj4o
2024-07-06 20:44:25.886855: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-07-06 20:44:25.886881: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp5gg1xj4o
2024-07-06 20:44:25.887561: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024-07-06 20:44:25.890089: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is n

78108

In [16]:
! xxd -i converted_model.tflite > ./src/model_data.cc

`now build and upload esp32 codes ...`

## udp test esp32 with test data

In [30]:
import socket
import json
import time

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_ip = '192.168.40.120'
server_port = 30000
sock.settimeout(5)

true_pred = 0
tutal = 0
for (_,x),y in zip(X_test.iterrows(),y_test):
    numbers = list(x)
    message = json.dumps(numbers).encode()
    sock.sendto(message, (server_ip, server_port))
    # print(f"Sent: {message}")
    response, server = sock.recvfrom(4096)
    response = json.loads(response.decode())
    expepted = int(y)
    recived = int(response['pred'])
    if expepted == recived:
        true_pred += 1
    tutal += 1
    print(f"expected:{expepted:2d}\trecived:{recived:2d}\taccuracy:{true_pred/tutal:.6f}")
sock.close()

expected:11	recived:11	accuracy:1.000000
expected: 1	recived: 1	accuracy:1.000000
expected:10	recived:10	accuracy:1.000000
expected: 1	recived: 1	accuracy:1.000000
expected: 8	recived: 8	accuracy:1.000000
expected: 9	recived: 9	accuracy:1.000000
expected: 7	recived: 7	accuracy:1.000000
expected: 0	recived: 0	accuracy:1.000000
expected: 6	recived: 6	accuracy:1.000000
expected:12	recived:12	accuracy:1.000000
expected:11	recived:11	accuracy:1.000000
expected:12	recived:12	accuracy:1.000000
expected: 1	recived: 1	accuracy:1.000000
expected:12	recived:12	accuracy:1.000000
expected:11	recived:11	accuracy:1.000000
expected: 2	recived: 2	accuracy:1.000000
expected: 9	recived: 9	accuracy:1.000000
expected:12	recived:12	accuracy:1.000000
expected: 5	recived: 5	accuracy:1.000000
expected: 4	recived: 4	accuracy:1.000000
expected:12	recived:12	accuracy:1.000000
expected: 8	recived: 8	accuracy:1.000000
expected: 7	recived: 7	accuracy:1.000000
expected: 1	recived: 1	accuracy:1.000000
expected: 3	reci

KeyboardInterrupt: 